# AI for Medicine — Patient Overlap and Data Leakage

Patient overlap in medical data is a part of a more general problem in machine learning called **data leakage**.  To identify patient overlap in this week's graded assignment, you'll check to see if a patient's ID appears in both the training set and the test set. You should also verify that you don't have patient overlap in the training and validation sets, which is what you'll do here.

Below is a simple example showing how you can check for and remove patient overlap in your training and validations sets.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
sns.set()

### Read in the data from a csv file

First, you'll read in your training and validation datasets from csv files. Run the next two cells to read these csvs into `pandas` dataframes.

In [7]:
train_df = pd.read_csv("/kaggle/input/chest-x-ray/train-small.csv")
train_df.head()

,Image,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,PatientId,Pleural_Thickening,Pneumonia,Pneumothorax
0,00008270_015.png,0,0,0,0,0,0,0,0,0,0,0,8270,0,0,0
1,00029855_001.png,1,0,0,0,1,0,0,0,1,0,0,29855,0,0,0
2,00001297_000.png,0,0,0,0,0,0,0,0,0,0,0,1297,1,0,0
3,00012359_002.png,0,0,0,0,0,0,0,0,0,0,0,12359,0,0,0
4,00017951_001.png,0,0,0,0,0,0,0,0,1,0,0,17951,0,0,0


In [8]:
train_df.shape

(1000, 16)

In [23]:
valid_df = pd.read_csv("/kaggle/input/chest-x-ray/valid-small.csv")
valid_df.head()

,Image,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,PatientId,Pleural_Thickening,Pneumonia,Pneumothorax
0,00027623_007.png,0,0,0,1,1,0,0,0,0,0,0,27623,0,0,0
1,00028214_000.png,0,0,0,0,0,0,0,0,0,0,0,28214,0,0,0
2,00022764_014.png,0,0,0,0,0,0,0,0,0,0,0,22764,0,0,0
3,00020649_001.png,1,0,0,0,1,0,0,0,0,0,0,20649,0,0,0
4,00022283_023.png,0,0,0,0,0,0,0,0,0,0,0,22283,0,0,0


In [24]:
valid_df.shape

(109, 16)

### Extract and compare the PatientId columns from the train and validation sets
By running the next four cells you will do the following:
1. Extract patient IDs from the train and validation sets
2. Convert these arrays of numbers into `set()` datatypes for easy comparison
3. Identify patient overlap in the intersection of the two sets

In [12]:
ids_train = train_df.PatientId.values
ids_valid = valid_df.PatientId.values

In [13]:
ids_train_set = set(ids_train)
print(f'There are {len(ids_train)} IDs but only {len(ids_train_set)} unique Patient IDs in the training set')
ids_valid_set = set(ids_valid)
print(f'There are {len(ids_valid)} IDs but only {len(ids_valid_set)} unique Patient IDs in the validation set')

There are 1000 IDs but only 928 unique Patient IDs in the training set
There are 109 IDs but only 97 unique Patient IDs in the validation set


In [16]:
patient_overlap = list(ids_train_set.intersection(ids_valid_set))
n_overlap = len(patient_overlap)
print(f'There are {n_overlap} Patient IDs in both the training and validation sets')
print(f'These patients are in both the training and validation datasets:')
patient_overlap

There are 11 Patient IDs in both the training and validation sets
These patients are in both the training and validation datasets:


[20290, 27618, 9925, 10888, 22764, 19981, 18253, 4461, 28208, 8760, 7482]

### Identify rows (indices) of overlapping patients and remove from either the train or validation set

These are the patient IDs that are on the validation set (13 counts):

In [42]:
valid_df[valid_df['PatientId'].isin(patient_overlap)]['PatientId'].value_counts()

PatientId
8760     2
28208    2
22764    1
10888    1
19981    1
7482     1
18253    1
9925     1
4461     1
27618    1
20290    1
Name: count, dtype: int64

In [44]:
valid_df = valid_df[~valid_df['PatientId'].isin(patient_overlap)]
valid_df

,Image,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,PatientId,Pleural_Thickening,Pneumonia,Pneumothorax
0,00027623_007.png,0,0,0,1,1,0,0,0,0,0,0,27623,0,0,0
1,00028214_000.png,0,0,0,0,0,0,0,0,0,0,0,28214,0,0,0
3,00020649_001.png,1,0,0,0,1,0,0,0,0,0,0,20649,0,0,0
4,00022283_023.png,0,0,0,0,0,0,0,0,0,0,0,22283,0,0,0
5,00003098_000.png,0,0,0,0,1,0,0,0,1,0,0,3098,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,00025335_000.png,0,0,0,0,0,0,0,0,0,0,0,25335,0,0,0
105,00019627_000.png,0,0,0,0,0,0,0,0,0,0,0,19627,0,0,0
106,00018464_017.png,0,0,0,1,0,0,0,0,1,0,0,18464,0,1,0
107,00009964_000.png,0,0,0,0,0,0,0,0,0,0,0,9964,0,0,0


In [43]:
# Alternative solution:

# 1. Create lists of the overlapping row numbers in both the training and validation sets. 
# 2. Drop the overlapping patient records from the validation set (could also choose to drop from train set)

# train_overlap_idxs = []
# valid_overlap_idxs = []

# for idx in range(n_overlap):
#     train_overlap_idxs.extend(train_df.index[train_df['PatientId'] == patient_overlap[idx]].tolist())
#     valid_overlap_idxs.extend(valid_df.index[valid_df['PatientId'] == patient_overlap[idx]].tolist())
    
# print(f'These are the indices of overlapping patients in the training set: ')
# print(f'{train_overlap_idxs}')
# print(f'These are the indices of overlapping patients in the validation set: ')
# print(f'{valid_overlap_idxs}')

# # Drop the overlapping rows from the validation set
# valid_df.drop(valid_overlap_idxs, inplace=True)

### Check that everything worked as planned by rerunning the patient ID comparison between train and validation sets.

When you run the next two cells you should see that there are now fewer records in the validation set and that the overlap problem has been removed!

In [45]:
ids_valid = valid_df.PatientId.values
ids_valid_set = set(ids_valid)
print(f'There are {len(ids_valid_set)} unique Patient IDs in the validation set')

There are 86 unique Patient IDs in the validation set


In [47]:
patient_overlap = list(ids_train_set.intersection(ids_valid_set))
n_overlap = len(patient_overlap)
print(f'There are {n_overlap} Patient IDs in both the training and validation sets')

There are 0 Patient IDs in both the training and validation sets
